# Library

In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import optuna


from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Data Loading

In [25]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [26]:
train

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience,blueWins
0,0,0,5,8,6,0,0,14536,17256,0
1,1,1,10,1,5,0,0,14536,17863,0
2,2,0,3,10,2,0,0,17409,17256,0
3,3,1,7,10,8,0,0,19558,18201,0
4,4,0,4,9,4,0,0,17409,17256,0
...,...,...,...,...,...,...,...,...,...,...
7995,9993,1,9,6,14,0,0,18513,18201,1
7996,9994,0,3,10,4,0,0,17381,19797,0
7997,9995,1,9,3,9,2,1,18274,18491,1
7998,9997,0,5,10,6,1,1,18274,18491,1


In [27]:
test

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience
0,9,0,7,6,6,0,0,16961,18201
1,15,0,6,6,6,2,1,18513,18021
2,18,1,6,4,3,0,0,13475,17256
3,23,0,5,4,7,0,0,17409,17256
4,31,0,10,8,9,0,0,18117,18472
...,...,...,...,...,...,...,...,...,...
1995,9971,0,6,3,7,0,0,17409,17256
1996,9980,0,4,4,4,0,0,17409,17256
1997,9983,1,6,3,3,1,1,18513,18201
1998,9996,1,10,9,9,1,1,18513,18201


# Feature

- 各特長量の説明

| ヘッダ名称            | データ型 | 説明                                                       |
|---------------------|-------|----------------------------------------------------------|
| gameId             | int   | ゲームID                                                    |
| blueFirstBlood      | int   | ゲームの最初のキル。青チームが最初のキルを行った場合は1、それ以外の場合は0 |
| blueKills           | int   | 青チームによって殺された敵の数                                     |
| blueDeaths          | int   | 青チームの死亡者数                                            |
| blueAssists         | int   | 青チームのキルアシストの数                                      |
| blueEliteMonsters   | int   | 青チームによって殺されたエリートモンスターの数（ドラゴンとヘラルド）       |
| blueDragons         | int   | 青チームによって殺されたドラゴンの数                               |
| blueTotalGold       | int   | 青チームの得たゴールド合計                                       |
| blueTotalExperience | int   | 青チームの得た経験値合計                                       |
| blueWins            | int   | 目的変数（青チームが勝った場合は1、それ以外の場合は0。）                |


# Feature Engineering

- Add new feature

In [28]:
def create_features(df):
    """
    与えられたデータフレームに対して特徴量エンジニアリングを行う関数。

    Args:
    df (DataFrame): 特徴量エンジニアリングを行うデータフレーム

    Returns:
    DataFrame: 新しい特徴量が追加されたデータフレーム
    """
    # キルとデスの比率
    df['killDeathRatio'] = df['blueKills'] / (df['blueDeaths'] + 1)  # ゼロ除算を防ぐために+1

    # アシストの割合
    df['assistRate'] = df['blueAssists'] / (df['blueKills'] + 1)

    # ヘラルドの討伐数
    df['blueHerald'] = df['blueEliteMonsters'] - df['blueDragons']

    # エリートモンスターとドラゴンの比率
    df['eliteMonsterDragonRatio'] = df['blueEliteMonsters'] / (df['blueDragons'] + 1)
    # エリートモンスターとヘラルドの比率
    df['eliteMonsterHeraldRatio'] = df['blueEliteMonsters'] / (df['blueHerald'] + 1)

    # ゴールドと経験値の対数変換
    df['logTotalGold'] = np.log1p(df['blueTotalGold'])
    df['logTotalExperience'] = np.log1p(df['blueTotalExperience'])

    # ゴールドと経験値の相互作用
    df['goldXexperience'] = df['blueTotalGold'] * df['blueTotalExperience']
    
    return df

- Apply data

In [29]:
train = create_features(train)
test = create_features(test)

# model

- LightGBM

In [30]:


# データの準備
X = train.drop('blueWins', axis=1)
y = train['blueWins']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# Optunaでのハイパーパラメータチューニング
def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'binary_error',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
    }

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

    verbose_eval = 0
    verbose = lgb.log_evaluation(verbose_eval)

    # 早期停止の設定
    early_stopping = lgb.early_stopping(stopping_rounds=10, verbose=True)
    model = lgb.train(param, lgb_train, num_boost_round=1000, valid_sets=[lgb_val], callbacks=[early_stopping, verbose])
    y_pred = model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = (y_pred > 0.5).astype(int)
    accuracy = accuracy_score(y_val, y_pred)

    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# 最適なハイパーパラメータでモデルを再学習
best_params = study.best_params
best_params['objective'] = 'binary'
best_params['metric'] = 'binary_error'
best_params['verbosity'] = -1

lgb_train = lgb.Dataset(X, y)
model = lgb.train(best_params, lgb_train, num_boost_round=study.best_trial.number)

# テストデータに対する予測
y_test_pred = model.predict(test)
y_test_pred_int = (y_test_pred > 0.5).astype(int)

# 予測結果の表示
print(y_test_pred_int)


[I 2024-01-17 11:49:10,405] A new study created in memory with name: no-name-60c2ffd7-8e66-4c57-834d-e397d6a61735


Training until validation scores don't improve for 10 rounds


[I 2024-01-17 11:49:10,939] Trial 0 finished with value: 0.804375 and parameters: {'lambda_l1': 0.00017624738908589123, 'lambda_l2': 1.314521679732859, 'num_leaves': 124, 'feature_fraction': 0.5635342262257073, 'bagging_fraction': 0.6429817274535997, 'bagging_freq': 1, 'min_child_samples': 59}. Best is trial 0 with value: 0.804375.
[I 2024-01-17 11:49:11,060] Trial 1 finished with value: 0.791875 and parameters: {'lambda_l1': 0.00967584315752473, 'lambda_l2': 8.936247966836342e-06, 'num_leaves': 72, 'feature_fraction': 0.4919130316281304, 'bagging_fraction': 0.4944569157819766, 'bagging_freq': 3, 'min_child_samples': 38}. Best is trial 0 with value: 0.804375.


Early stopping, best iteration is:
[45]	valid_0's binary_error: 0.195625
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[21]	valid_0's binary_error: 0.208125
Training until validation scores don't improve for 10 rounds


[I 2024-01-17 11:49:11,176] Trial 2 finished with value: 0.804375 and parameters: {'lambda_l1': 5.647284739516784e-08, 'lambda_l2': 0.10870273192407653, 'num_leaves': 22, 'feature_fraction': 0.8036250542582613, 'bagging_fraction': 0.40803139182262405, 'bagging_freq': 5, 'min_child_samples': 51}. Best is trial 0 with value: 0.804375.
[I 2024-01-17 11:49:11,287] Trial 3 finished with value: 0.805625 and parameters: {'lambda_l1': 2.7250975553601304e-08, 'lambda_l2': 9.7090131286385e-07, 'num_leaves': 159, 'feature_fraction': 0.9146531391285586, 'bagging_fraction': 0.8538850536497055, 'bagging_freq': 3, 'min_child_samples': 91}. Best is trial 3 with value: 0.805625.
[I 2024-01-17 11:49:11,348] Trial 4 finished with value: 0.799375 and parameters: {'lambda_l1': 3.622613942656888, 'lambda_l2': 3.723663473696467, 'num_leaves': 24, 'feature_fraction': 0.7782517314399702, 'bagging_fraction': 0.8999454541822157, 'bagging_freq': 3, 'min_child_samples': 16}. Best is trial 3 with value: 0.805625.


...

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[41]	valid_0's binary_error: 0.190625
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_error: 0.216875


[I 2024-01-17 11:49:23,869] Trial 97 finished with value: 0.7925 and parameters: {'lambda_l1': 2.0879208828619884e-05, 'lambda_l2': 2.5285838882922413e-05, 'num_leaves': 160, 'feature_fraction': 0.5430742270168446, 'bagging_fraction': 0.8742010356845074, 'bagging_freq': 5, 'min_child_samples': 89}. Best is trial 49 with value: 0.814375.
[I 2024-01-17 11:49:24,027] Trial 98 finished with value: 0.803125 and parameters: {'lambda_l1': 1.535302800387085e-06, 'lambda_l2': 0.0001881032921221775, 'num_leaves': 203, 'feature_fraction': 0.4163065722258043, 'bagging_fraction': 0.9373236516841488, 'bagging_freq': 4, 'min_child_samples': 100}. Best is trial 49 with value: 0.814375.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[12]	valid_0's binary_error: 0.2075
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[33]	valid_0's binary_error: 0.196875


[I 2024-01-17 11:49:24,209] Trial 99 finished with value: 0.804375 and parameters: {'lambda_l1': 3.949605374603884e-06, 'lambda_l2': 0.0007845597414152589, 'num_leaves': 131, 'feature_fraction': 0.4757035549952813, 'bagging_fraction': 0.8402071222281411, 'bagging_freq': 5, 'min_child_samples': 17}. Best is trial 49 with value: 0.814375.


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[31]	valid_0's binary_error: 0.195625
[1 1 0 ... 1 1 0]


- testにy_test_pred結合

In [31]:
output = '../data/output/submit_cv_oputuna.csv'

In [32]:
submit = pd.DataFrame({
    '0': test['gameId'],
    '1': y_test_pred_int
})

# カラムヘッダー消去して出力
# ヘッダーなしでCSVファイルとして保存
submit.to_csv(output, header=False, index=False)

- 出力csvチェック

In [33]:
check = pd.read_csv(output)

In [34]:
check

,9,1
0,15,1
1,18,0
2,23,0
3,31,1
4,32,1
...,...,...
1994,9971,0
1995,9980,0
1996,9983,1
1997,9996,1
